Монтирование диска Google Drive для загрузк и харанения  данных.

**ВНИМАНИЕ** для доступа к данным необходимо выполнить следующие действия:

1. Перейти по [ссылке](https://drive.google.com/drive/folders/1MnoxVXjG8o8kL0SS14E3YzbZmCovsGpR?usp=sharing)
2. Создать ярлык на данную директорию в своём Google Drive. Для этого нажмите на название директории а затем на "Добавить ярлык на Диск".

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
from pathlib import Path
prefix = Path("/content/gdrive/My Drive/Kononov_NLP")

Установка необходимых библиотек в окружение

In [ ]:
!pip install deeppavlov
# !python -m deeppavlov install "/content/gdrive/My Drive/Kononov_NLP/ELMo/elmo_ru_wiki/elmo_ru_wiki.json"
!python -m deeppavlov install "/content/gdrive/My Drive/Kononov_NLP/ELMo/elmo_ru_tw/elmo_ru_tw.json"
!pip install tensorflow-hub==0.1.1
!pip install tensorflow-gpu==1.14.0
!pip install pandas

Импорт необходимых компонентов

In [ ]:
from deeppavlov import build_model
from deeppavlov import configs
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder
from tqdm import tqdm

Выбор и загрузка модели для получения эмбеддингов

In [ ]:
# name, alias = "elmo_ru_tw", "twitter"
# name, alias = "elmo_ru_wiki", "wiki"
name, alias = "elmo_ru_news", "news"
model = ELMoEmbedder(str(prefix / "ELMo/{}".format(name)))

Проверка работы на простом примере

In [ ]:
%%time
text = [["мама", "мыла", "раму", "папе"], ]
emb = model(text)

Подсчёт близости запросов и заголовков документов в пространстве эмбеддингов.

In [ ]:
queries_df = pd.read_csv(prefix / "queries.tsv", sep="\t", index_col=0)
titles_df = pd.read_csv(prefix / "titles.tsv", sep="\t", index_col=0).fillna("")
titles_df["text"] = titles_df["text"].apply(lambda x: " ".join(x.split()[:50]))

relation_df = pd.read_csv(prefix / "samples.tsv", sep="\t")

relation_df.drop(columns=["label", "sample_id"], inplace=True)
relation_df = relation_df.assign(embedding_similarity=np.full(relation_df.index.size, 0.0))
for key, group in tqdm(relation_df.groupby("query_id")):
  doc_idx = group["doc_id"].values
  query_embedding = model([queries_df["text"].loc[key].split(), ])
  title_embeddings = model(titles_df["text"].loc[doc_idx].apply(str.split).to_list())
  relation_df.loc[group.index, "embedding_similarity"] = cosine_similarity(query_embedding.reshape(1, -1),
                                                                           title_embeddings).reshape(-1)

path = prefix / "semantic/elmo_{}_query_doc_embedding_similarity.tsv".format(alias)
relation_df.to_csv(path, sep="\t", index=False)
print("Result saved to file: {}".format(path))